In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab4").getOrCreate()
sc=spark.sparkContext

textFiles = sc.textFile('hdfs://localhost:9000//user/student/gutenberg/*')

textFiles.first()


22/12/10 13:45:52 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/10 13:45:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/10 13:45:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/10 13:45:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/10 13:45:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/10 13:45:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


22/12/10 13:46:13 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:191)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3033)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:829)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:754)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'The Project Gutenberg eBook of Ulysses, by James Joyce'

In [32]:
words = textFiles\
    .flatMap(lambda line: line.split(" "))



counts = words\
    .map(lambda word: (word, 1))\
    .filter(lambda word: len(word[0]) >= 5)\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[1],word[0]))\
    .sortByKey(False)

counts.take(6)

[(3637, 'which'),
 (1691, 'their'),
 (1224, 'there'),
 (954, 'other'),
 (884, 'would'),
 (846, 'these')]

In [37]:
bigrams = textFiles.flatMap(lambda line: line.split("."))\
    .map(lambda line: line.strip().split(" "))\
    .flatMap(lambda words: (tuple(word) for word in zip(words, words[1: ])))

pair_counts = bigrams\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[1],word[0]))\
    .sortByKey(False)

pair_counts.take(7)

[(10929, ('', '')),
 (6900, ('of', 'the')),
 (3501, ('in', 'the')),
 (2208, ('to', 'the')),
 (1637, ('and', 'the')),
 (1618, ('on', 'the')),
 (1231, ('from', 'the'))]

In [43]:
bigrams1 = textFiles.flatMap(lambda line: line.split("-"))\
        .flatMap(lambda line: line.split(" "))\
        .flatMap(lambda line: line.split("—"))\


counts1 = words\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[0]))\
    .sortBy(lambda word: len(word) * -1)

counts1.take(1)

['Nationalgymnasiummuseumsanatoriumandsuspensoriumsordinaryprivatdocentge']

In [51]:
bigrams2  = textFiles\
    .filter(lambda word: len(word) > 0 and word[0].isalpha())\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y)

counts2 = bigrams2.keys()\
    .map(lambda word: (word[0].upper(), 1)) \
    .reduceByKey(lambda x, y: x + y)\
    .sortByKey()

counts3.take(6)

[('A', 4943), ('B', 2990), ('C', 2675), ('D', 1843), ('E', 1405), ('F', 2148)]

In [59]:
from pyspark.sql.functions import col
from pyspark.sql.functions import max

weatherLines = sc.textFile('hdfs://localhost:9000/user/student/weather/*')
weatherColumns = ["year","temperature", "quality"]
weatherDF = weatherLines\
    .map(lambda val: (int(val[15:19]), int(val[87:92]), int(val[92:93])))\
    .toDF(weatherColumns)

weatherDF.createOrReplaceTempView("Weather")

sqlWay = spark.sql("""
    SELECT year, max(temperature) FROM Weather
    WHERE quality IN (0,1,4,5,9) AND temperature != 9999
    GROUP BY year
""")

dataFrameWay = weatherDF\
    .where(weatherDF.quality.isin([0, 1, 4, 5, 9]) & (weatherDF.temperature != 9999))\
    .groupBy(weatherDF.year)\
    .agg(max(weatherDF.temperature))

print(sqlWay.rdd.map(lambda x: (x[0], x[1])).collect())
print(dataFrameWay.rdd.map(lambda x: (x[0], x[1])).collect())

[(2022, 320)]


[(2022, 320)]
